In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc, roc_auc_score

import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMClassifier
import gc
from sklearn.grid_search import GridSearchCV
from bayes_opt import BayesianOptimization
import time
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# load preprocessed data 1

file_name = "./data/train_preprocessed1.csv"
train_df1 = pd.read_csv(file_name, low_memory = False, index_col = False)

train_df1.head()



,Unnamed: 0,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
0,0,1,0.0,0.0,1.0,280199.0,100.0,0.0,0.0,0.0,...,3073.0,31.0,0.0,1.0,0.0,0.0,4.0,2.0,0.0,0.0
1,1,1,2.0,1.0,2.0,280103.0,30.0,280106.0,30.0,280203.0,...,2538.0,25.0,1.0,2.0,0.0,0.0,6.0,12.0,2.0,2.0
2,2,1,29.0,2.0,1.0,321004.0,60.0,321216.0,40.0,0.0,...,2923.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,5.0,2.0
3,3,1,40.0,2.0,3.0,270602.0,50.0,320602.0,50.0,0.0,...,2678.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,13.0,3.0
4,4,0,59.0,1.0,1.0,260500.0,34.0,280000.0,33.0,290000.0,...,2153.0,19.0,1.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0


In [4]:
train_df1.drop(train_df1.columns[0], axis = 1, inplace = True)
train_df1.head()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
0,1,0.0,0.0,1.0,280199.0,100.0,0.0,0.0,0.0,0.0,...,3073.0,31.0,0.0,1.0,0.0,0.0,4.0,2.0,0.0,0.0
1,1,2.0,1.0,2.0,280103.0,30.0,280106.0,30.0,280203.0,40.0,...,2538.0,25.0,1.0,2.0,0.0,0.0,6.0,12.0,2.0,2.0
2,1,29.0,2.0,1.0,321004.0,60.0,321216.0,40.0,0.0,0.0,...,2923.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,5.0,2.0
3,1,40.0,2.0,3.0,270602.0,50.0,320602.0,50.0,0.0,0.0,...,2678.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,13.0,3.0
4,0,59.0,1.0,1.0,260500.0,34.0,280000.0,33.0,290000.0,33.0,...,2153.0,19.0,1.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0


In [5]:
file_name = "./data/train_preprocessed2.csv"
train_df2 = pd.read_csv(file_name, low_memory = False)

train_df2.head()

,A..papers,A.papers,B.papers,C.papers,Dif.countries,Perc_non_australian,Number.people,PHD,Max.years.univ,Grants.succ,...,SEO.11,SEO.12,SEO.13,SEO.14,SEO.15,SEO.16,SEO.17,SEO.18,SEO.19,Grant.Status
0,4.0,2.0,0.0,0.0,1,0.00,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,6.0,12.0,2.0,2.0,1,1.00,1,1.0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,7.0,20.0,20.0,7.0,2,0.75,4,2.0,50.0,0.0,...,0,0,2,0,0,0,0,0,0,1
3,0.0,3.0,13.0,3.0,1,1.00,2,2.0,15.0,0.0,...,0,0,2,0,0,0,0,0,0,1
4,3.0,0.0,1.0,0.0,1,0.00,1,1.0,10.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
#Setup data : Divide Data and Target

data_df1 = train_df1.drop(['Grant.Status'], axis = 1)
target_df1 = train_df1['Grant.Status']

data_df2 = train_df2.drop(['Grant.Status'], axis = 1)
target_df2 = train_df2['Grant.Status']

data1 = data_df1.values
target1 = target_df1.values
data2 = data_df2.values
target2 = target_df2.values

In [20]:
cnt = 0

In [ ]:
#parameter, results saved to drow plot later

max_depth_GS = np.zeros(1000)
min_child_weight_GS = np.zeros(1000)
colsample_bytree_GS = np.zeros(1000)
subsample_GS = np.zeros(1000)
gamma_GS = np.zeros(1000)
auc_GS = np.zeros(1000)

In [16]:
max_depth_BO = np.zeros(205)
min_child_weight_BO = np.zeros(205)
colsample_bytree_BO = np.zeros(205)
subsample_BO = np.zeros(205)
gamma_BO = np.zeros(205)
auc_BO = np.zeros(205)

In [24]:
max_depth_BO2 = np.zeros(205)
min_child_weight_BO2 = np.zeros(205)
colsample_bytree_BO2 = np.zeros(205)
subsample_BO2 = np.zeros(205)
gamma_BO2 = np.zeros(205)
auc_BO2 = np.zeros(205)

In [25]:
max_depth_DE = np.zeros(1500)
min_child_weight_DE = np.zeros(1500)
colsample_bytree_DE = np.zeros(1500)
subsample_DE = np.zeros(1500)
gamma_DE = np.zeros(1500)
auc_DE = np.zeros(1500)

In [26]:
max_depth_DE2 = np.zeros(1500)
min_child_weight_DE2 = np.zeros(1500)
colsample_bytree_DE2 = np.zeros(1500)
subsample_DE2 = np.zeros(1500)
gamma_DE2 = np.zeros(1500)
auc_DE2 = np.zeros(1500)

In [ ]:
#We will be using only data2 from now.
#data1 will be used after optimizing range is expanded to preprocess part.


In [8]:
#XGB Result (using Default Parameter)

model = xgb.XGBClassifier(eval_metric = 'auc')

# make predictions with kfold cross validation score
kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
results = cross_val_score(model, data2, target2, cv = kfold)
accuracy = results.mean()*100
print("Accuracy : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    

Accuracy : 87.93% (0.66%)


In [9]:
#LGB Result (using Default Parameter)

#LGB's default parameter is slightly different from xgb's 
#So several parameters should be setted differently according to xgb's

lgb_train = lgb.Dataset(data2, target2)
lgb_params = {
    'task': 'train',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'max_depth' : 6,
    'learning_rate' : 0.03,
    'reg_lambda' : 1.0
}
model = lgb.LGBMClassifier(**lgb_params)
    
kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
results = cross_val_score(model, data2, target2, cv = kfold)
auc = results.mean()*100
print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

AUC : 88.23% (1.01%)


In [ ]:
#XGB Result (using GridSearch, Optimized Parameter)



In [ ]:
#LGB Result (using GridSearch, Optimized Parameter)

In [ ]:
#XGB Train Model using BayesOpt

def XGB_Train_Model_BO(min_child_weight, max_depth, gamma, subsample, colsample_bytree) : 
    xgb_params = {
        #static parameters
        'n_trees' : 20,
        'eta' : 0.3,
        'objective' : 'reg:linear', 
        'eval_metric' : 'auc',
        'silent' : 1,
        
        #tuned parameters
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    
    model = xgb.XGBClassifier(**xgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data2, target2, cv = kfold)
    auc = results.mean()*100
    
    ##############################plot parameter saving part#########################################
    global cnt
    global max_depth_BO, subsample_BO, min_child_weight_BO, gamma_BO, colsample_bytree_BO, auc_BO
    max_depth_BO[cnt]        = max_depth
    subsample_BO[cnt]        = subsample
    min_child_weight_BO[cnt] = min_child_weight
    gamma_BO[cnt]            = gamma
    subsample_BO[cnt]        = subsample
    auc_BO[cnt]              = auc
    cnt = cnt + 1
    ##############################plot parameter saving part#########################################    
    
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    
    return auc


In [23]:
#XGB Result (using BayesOpt, Optimized Parameter)

xgb_params = {
    
    #Minimum sum of weights : to control overfitting
    'min_child_weight' : (1, 20), 
    
    #Maximum depth of a tree : to control overfitting
    'max_depth' : (2, 10),
    
    #minimum loss reduction required to make a split : makes algorithm conservative
    'gamma' : (0, 10), 
    
    #Fraction of observations to be randomly samples for each tree
    #Lower: prevent overfitting
    'subsample' : (0.5, 1),
    
    #Fraction of columns to be randomly samples for each tree
    'colsample_bytree' : (0.1, 1),
    
    }


xgb_bayesOPT = BayesianOptimization(XGB_Train_Model_BO, xgb_params)
start_time = time.time()
xgb_bayesOPT.maximize(init_points = 5, n_iter = 200)
elapsed_time = time.time() - start_time
print("elapsed time : %s min %s sec"%elapsed_time/60, elapsed_time%60)
cnt = 0

Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
AUC : 86.94% (1.06%)
    1 | 00m05s |   86.94316 |             0.4556 |    8.3939 |      8.1478 |            16.7161 |      0.9308 | 
AUC : 86.68% (1.23%)
    2 | 00m07s |   86.67904 |             0.5928 |    9.5919 |      9.3732 |            18.9426 |      0.9196 | 
AUC : 88.75% (1.07%)
    3 | 00m05s |   88.74603 |             0.8407 |    0.9763 |      5.8055 |            14.4060 |      0.8505 | 
AUC : 86.71% (0.71%)
    4 | 00m02s |   86.71350 |             0.5186 |    4.2265 |      3.4129 |             8.0669 |      0.6426 | 
AUC : 84.82% (1.21%)
    5 | 00m01s |   84.81873 |             0.3255 |    9.0580 |      2.4391 |             6.3987 |      0.9898 | 
Bayesian Optimization
--------------------------------------------------------------------

AUC : 89.65% (0.38%)
   56 | 00m18s |   89.65320 |             0.2373 |    0.4407 |      8.4230 |            14.0724 |      0.8847 | 
AUC : 88.88% (0.94%)
   57 | 00m26s |   88.88380 |             1.0000 |    0.2441 |      6.5048 |            14.1310 |      0.5000 | 
AUC : 86.13% (0.91%)
   58 | 00m17s |   86.12780 |             0.7361 |    4.6321 |      3.6671 |             9.8670 |      0.8622 | 
AUC : 89.66% (0.84%)
   59 | 00m21s |   89.66478 |             0.6105 |    0.3526 |      7.9118 |            14.0510 |      0.8050 | 
AUC : 89.94% (0.50%)
   60 | 00m19s |   89.94031 |             0.3119 |    0.0000 |      8.3759 |            14.4908 |      1.0000 | 
AUC : 89.94% (0.40%)
   61 | 00m18s |   89.94030 |             0.3218 |    0.1256 |      8.6230 |            14.5262 |      0.8605 | 
AUC : 89.75% (0.67%)
   62 | 00m20s |   89.74508 |             0.2536 |    0.1691 |      8.4091 |            14.6010 |      0.9257 | 
AUC : 89.91% (0.73%)
   63 | 00m20s |   89.90590 |            

AUC : 89.76% (0.52%)
  115 | 00m27s |   89.75663 |             0.4133 |    0.1900 |      8.2119 |            13.9424 |      0.7745 | 
AUC : 89.85% (0.55%)
  116 | 00m25s |   89.84844 |             0.4086 |    0.3267 |      8.0250 |            13.9945 |      1.0000 | 
AUC : 88.73% (0.91%)
  117 | 00m27s |   88.73455 |             1.0000 |    0.7285 |      6.1626 |            19.7964 |      0.8870 | 
AUC : 89.38% (1.14%)
  118 | 00m28s |   89.37758 |             0.5623 |    0.2705 |      7.5418 |            15.3459 |      0.6159 | 
AUC : 89.30% (0.40%)
  119 | 00m24s |   89.29721 |             0.2820 |    0.3795 |      7.5176 |            14.4193 |      0.5000 | 
AUC : 89.43% (0.78%)
  120 | 00m29s |   89.43498 |             0.6873 |    0.0636 |      7.0062 |            16.1153 |      0.6806 | 
AUC : 89.56% (0.90%)
  121 | 00m26s |   89.56142 |             0.2224 |    0.2025 |      8.3490 |            14.1447 |      0.7549 | 
AUC : 89.55% (0.61%)
  122 | 00m27s |   89.54988 |            

KeyboardInterrupt: 

In [ ]:
#LGB Train Model

def LGB_Train_Model_BO(gamma, max_depth, min_child_weight, colsample_bytree, subsample) :
    lgb_train = lgb.Dataset(data, target)
    
    lgb_params = {
        
    #static parameters
    'task': 'train',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'learning_rate' : 0.03,
    'reg_lambda' : 1.0,
    'num_leaves' : 1023,
        
    #tuned parameters
    'max_depth': int(max_depth),
    'min_child_weight' : int(min_child_weight),
    'colsample_bytree' : max(min(colsample_bytree, 1), 0),
    'subsample' : max(min(subsample, 1), 0),
    'gamma' : max(gamma, 0), 
    }

    model = lgb.LGBMClassifier(**lgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data2, target2, cv = kfold)
    auc = results.mean()*100
    
    ##############################plot parameter saving part#########################################
    global cnt, optimizer
    global max_depth_BO2, subsample_BO2, min_child_weight_BO2, gamma_BO2, colsample_bytree_BO2, auc_BO2
    max_depth_BO2[cnt]        = max_depth
    subsample_BO2[cnt]        = subsample
    min_child_weight_BO2[cnt] = min_child_weight
    gamma_BO2[cnt]            = gamma
    subsample_BO2[cnt]        = subsample
    auc_BO2[cnt]              = auc
    cnt = cnt + 1
    ##############################plot parameter saving part#########################################
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    
    return auc

In [ ]:
#LGB Result (using BayesOpt, Optimized Parameter)
lgb_params = {
    'max_depth' : (2, 10), 
    'min_child_weight' : (1, 20), 
    'colsample_bytree' : (0.1, 1), 
    'subsample' : (0.5, 1),
    'gamma' : (0, 10)
}

lgb_bayesOPT = BayesianOptimization(LGB_Train_Model_BO, lgb_params)
start_time = time.time()
lgb_bayesOPT.maximize(init_points = 5, n_iter = 200)
elapsed_time = time.time() - start_time
print("elapsed time : %s min %s sec"%elapsed_time/60, elapsed_time%60)
cnt = 0

In [ ]:
#XGB Train Model

def XGB_Train_Model_DE(params) : 
    min_child_weight = params[0]
    max_depth = params[1]
    gamma = params[2]
    subsample = params[3] 
    colsample_bytree = params[4]
    xgb_params = {
        #static parameters
        'n_trees' : 20,
        'eta' : 0.3,
        'objective' : 'reg:linear', 
        'eval_metric' : 'auc',
        'silent' : 1,
        
        #tuned parameters
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    
    model = xgb.XGBClassifier(**xgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data2, target2, cv = kfold)
    auc = results.mean()*100
    
    ##############################plot parameter saving part#########################################
    global cnt
    global max_depth_DE, subsample_DE, min_child_weight_DE, gamma_DE, colsample_bytree_DE, auc_DE
    max_depth_DE[cnt]        = max_depth
    subsample_DE[cnt]        = subsample
    min_child_weight_DE[cnt] = min_child_weight
    gamma_DE[cnt]            = gamma
    subsample_DE[cnt]        = subsample
    auc_DE[cnt]              = auc
    cnt = cnt + 1
    ##############################plot parameter saving part#########################################    
    
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    return 100 - auc


In [ ]:
#XGB Result (using DiffEvolution, Optimized Parameter)

bounds = [(1,20), (2, 10), (0, 10), (0.5, 1), (0.1, 1)]
start_time = time.time()
result = differential_evolution(XGB_Train_Model_DE, bounds, maxiter = 10, popsize = 30)
elapsed_time = time.time() - start_time
print("elapsed time : %s min %s sec"%elapsed_time/60, elapsed_time%60)
cnt = 0

result.x, result.fun

In [ ]:
#LGB Train Model

def LGB_Train_Model_DE(params) :
    min_child_weight = params[0]
    max_depth = params[1]
    gamma = params[2]
    subsample = params[3] 
    colsample_bytree = params[4]
    lgb_train = lgb.Dataset(data, target)
    
    lgb_params = {
        
    #static parameters
    'task': 'train',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'learning_rate' : 0.03,
    'reg_lambda' : 1.0,
    'num_leaves' : 1023,
        
    #tuned parameters
    'max_depth': int(max_depth),
    'min_child_weight' : int(min_child_weight),
    'colsample_bytree' : max(min(colsample_bytree, 1), 0),
    'subsample' : max(min(subsample, 1), 0),
    'gamma' : max(gamma, 0), 
    }

    model = lgb.LGBMClassifier(**lgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data2, target2, cv = kfold)
    auc = results.mean()*100
    
    ##############################plot parameter saving part#########################################
    global cnt
    global max_depth_DE2, subsample_DE2, min_child_weight_DE2, gamma_DE2, colsample_bytree_DE2, auc_DE2
    max_depth_DE2[cnt]        = max_depth
    subsample_DE2[cnt]        = subsample
    min_child_weight_DE2[cnt] = min_child_weight
    gamma_DE2[cnt]            = gamma
    subsample_DE2[cnt]        = subsample
    auc_DE2[cnt]              = auc
    cnt = cnt + 1
    ##############################plot parameter saving part#########################################
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    return 100 - auc

In [27]:
#LGB Result (using DiffEvolution, Optimized Parameter)
bounds = [(1,20), (2, 10), (0, 10), (0.5, 1), (0.1, 1)]
start_time = time.time()
result = differential_evolution(LGB_Train_Model_DE, bounds, maxiter = 10, popsize = 30)
elapsed_time = time.time() - start_time
print("elapsed time : %s min %s sec"%elapsed_time/60, elapsed_time%60)
cnt = 0

result.x, result.fun